In [1]:
from uagents import Agent, Context
from openai import OpenAI
import ast
import re

In [2]:
OPENAI_KEY = open(".env", "r").read().strip()

client = OpenAI(api_key=OPENAI_KEY)


def message(role, content):
  return {
    "role": role,
    "content": content
  }

def chat(messages):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
  )
  response =  completion.choices[0].message.content
#   messages.append(message("assistant", response))
  return response

In [ ]:
Questionnaire_agent = Agent(name="questionnaire_agent", seed="questionnaire_agent_seed")

@Questionnaire_agent.on_event("startup")
async def questionnaire_agent(ctx: Context):
    ctx.logger.info("Questionnaire agent is kicking off")

    output_struct = {"questions":["list of questions about the 3D model to be built, along with sample options"]}

    chat_msgs = []

    SYSTEM_PROMPT = f"""You're a 3D graphic designer & planner who specializes in creating 3D models schema based on descriptions of objects. Do not assume anything, you have to ask questions about the model, like how it has to be, its approx dimensions, texture, features and other user customizations too. return the response in this json format: {output_struct} only!, don't include any introductory/ending text."""

    chat_msgs.append(message("system", SYSTEM_PROMPT))
    chat_msgs.append(message("user", "I want a house"))

    print(ctx.logger.info(chat(chat_msgs)))

    try:
        questions = ast.literal_eval(chat(chat_msgs))
        print(questions)

        questions = [q for q in questions["questions"][0]]

        for q in questions:
            chat_msgs.append(message("system", q))
            chat_msgs.append(message("user", "#######"))
    except Exception as e:
        ctx.logger.error(e)
        print(e)
        questions = []

    ctx.logger.info(chat(chat_msgs))

In [23]:
output_struct = {"questions":["list of questions about the 3D model to be built, along with sample options"]}

chat_msgs = []

SYSTEM_PROMPT = f"""You're a 3D graphic designer & planner who specializes in creating 3D models schema based on descriptions of objects. Do not assume anything, you have to ask questions about the model, like how it has to be, its approx dimensions, texture, features and other user customizations too. return the response in this json format: {output_struct} only!, don't include any introductory/ending text."""

chat_msgs.append(message("system", SYSTEM_PROMPT))
chat_msgs.append(message("user", "I want a rectangular block"))


questions = chat(chat_msgs)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
print(questions)

{
    'questions': [
        {
            'question': 'What are the dimensions of the rectangular block?',
            'options': ['Length', 'Width', 'Height']
        },
        {
            'question': 'What material should the rectangular block be made of?',
            'options': ['Wood', 'Metal', 'Plastic', 'Stone', 'Other']
        },
        {
            'question': 'What texture should the surface of the rectangular block have?',
            'options': ['Smooth', 'Rough', 'Glossy', 'Matte']
        },
        {
            'question': 'What color should the rectangular block be?',
            'options': ['Specify Color']
        },
        {
            'question': 'Should there be any specific features on the block?',
            'options': ['Holes', 'Etchings', 'Grooves', 'Other', 'None']
        },
        {
            'question': 'Is there any particular weight requirement for the block?',
            'options': ['Specify Weight']
        }
    ]
}


In [25]:
try:
	if re.search(r"```json", questions):
		questions = re.sub(r"```json", "", questions)
		questions = re.search(r"```", questions)
		qu = ast.literal_eval(questions)
	else:
		qu = ast.literal_eval(questions)
		
	qu = [q for q in qu["questions"]]

	for q in qu:
		print(q)
except Exception as e:
	print(e)
	qu = []

{'question': 'What are the dimensions of the rectangular block?', 'options': ['Length', 'Width', 'Height']}
{'question': 'What material should the rectangular block be made of?', 'options': ['Wood', 'Metal', 'Plastic', 'Stone', 'Other']}
{'question': 'What texture should the surface of the rectangular block have?', 'options': ['Smooth', 'Rough', 'Glossy', 'Matte']}
{'question': 'What color should the rectangular block be?', 'options': ['Specify Color']}
{'question': 'Should there be any specific features on the block?', 'options': ['Holes', 'Etchings', 'Grooves', 'Other', 'None']}
{'question': 'Is there any particular weight requirement for the block?', 'options': ['Specify Weight']}
